# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib as plt
from sklearn import model_selection

In [98]:
data = pd.read_csv('California Housing data.csv')
data = data.dropna(axis=0)

In [99]:
data.head()

,Unnamed: 0,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedInc,Latitude,Longitude,medianHouseValue
0,0,41.0,6.984127,1.023810,322.0,2.555556,8.3252,37.88,-122.23,4.526
1,1,21.0,6.238137,0.971880,2401.0,2.109842,8.3014,37.86,-122.22,3.585
2,2,52.0,8.288136,1.073446,496.0,2.802260,7.2574,37.85,-122.24,3.521
3,3,52.0,5.817352,1.073059,558.0,2.547945,5.6431,37.85,-122.25,3.413
4,4,52.0,6.281853,1.081081,565.0,2.181467,3.8462,37.85,-122.25,3.422


In [100]:
data.drop(labels='Unnamed: 0', inplace=True, axis=1)
data.head().describe()

,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedInc,Latitude,Longitude,medianHouseValue
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,43.600000,6.721921,1.044655,868.400000,2.439414,6.674660,37.858000,-122.238000,3.693400
std,13.501852,0.970532,0.046619,862.336535,0.288132,1.921878,0.013038,0.013038,0.470897
min,21.000000,5.817352,0.971880,322.000000,2.109842,3.846200,37.850000,-122.250000,3.413000
25%,41.000000,6.238137,1.023810,496.000000,2.181467,5.643100,37.850000,-122.250000,3.422000
50%,52.000000,6.281853,1.073059,558.000000,2.547945,7.257400,37.850000,-122.240000,3.521000
75%,52.000000,6.984127,1.073446,565.000000,2.555556,8.301400,37.860000,-122.230000,3.585000
max,52.000000,8.288136,1.081081,2401.000000,2.802260,8.325200,37.880000,-122.220000,4.526000


In [101]:
y_data = data['medianHouseValue']
data.drop(labels=['medianHouseValue'], inplace=True, axis =1)
x_data = data

In [104]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(x_data, y_data, test_size=0.3, random_state=1)

In [105]:
y_train.head()

5748     2.387
5592     1.711
2110     0.554
13901    0.696
18425    2.131
Name: medianHouseValue, dtype: float64

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [106]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [107]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [108]:
data.columns

Index(['HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'MedInc',
       'Latitude', 'Longitude'],
      dtype='object')

In [109]:
X_train = pd.DataFrame(data=X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(data=X_test_scaled, columns=X_test.columns, index=X_test.index)

In [110]:
house_age = tf.feature_column.numeric_column('HouseAge')
av_rooms = tf.feature_column.numeric_column('AveRooms')
av_bedroom = tf.feature_column.numeric_column('AveBedrms')
population = tf.feature_column.numeric_column('Population')
av_occup = tf.feature_column.numeric_column('AveOccup')
med_income = tf.feature_column.numeric_column('MedInc')
longitude = tf.feature_column.numeric_column('Longitude')
latitude = tf.feature_column.numeric_column('Latitude')

feat_columns = [house_age, av_rooms, av_bedroom, population, av_occup, med_income, longitude, latitude]
feat_columns

[_NumericColumn(key='HouseAge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='AveRooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='AveBedrms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='AveOccup', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='MedInc', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Longitude', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Latitude', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [111]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10,num_epochs=1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [112]:
model = tf.estimator.DNNRegressor(hidden_units=[10, 12, 14, 12, 10], feature_columns=feat_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\aaleshin\\AppData\\Local\\Temp\\tmp7imrb_5e', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002D044F33278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [113]:
model.train(input_fn = input_func, steps = 50000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\aaleshin\AppData\Local\Temp\tmp7imrb_5e\model.ckpt.
INFO:tensorflow:loss = 77.32179, step = 1
INFO:tensorflow:global_step/sec: 223.215
INFO:tensorflow:loss = 16.766003, step = 101 (0.456 sec)
INFO:tensorflow:global_step/sec: 294.116
INFO:tensorflow:loss = 8.305816, step = 201 (0.332 sec)
INFO:tensorflow:global_step/sec: 277.782
INFO:tensorflow:loss = 10.156124, step = 301 (0.360 sec)
INFO:tensorflow:global_step/sec: 328.943
INFO:tensorflow:loss = 4.4904804, step = 401 (0.304 sec)
INFO:tensorflow:global_step/sec: 352.117
INFO:tensorflow:loss = 4.7010055, step = 501 (0.284 sec)
INFO:tensorflow:global_step/sec: 342.457
INFO:tensorflow:loss = 2.344146, step = 601 (0.292 sec)
INFO:tensorflow:global_step/

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [114]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=20,num_epochs=1,shuffle=False)

In [115]:
pred_model = model.predict(pred_input_func)
preds = list(pred_model)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\aaleshin\AppData\Local\Temp\tmp7imrb_5e\model.ckpt-50000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [116]:
final_y_preds = []
for pred in preds:
    final_y_preds.append(pred['predictions'])

In [117]:
final_y_preds[:20]

[array([2.4523005], dtype=float32),
 array([0.63539267], dtype=float32),
 array([2.9016871], dtype=float32),
 array([1.8752469], dtype=float32),
 array([3.0156007], dtype=float32),
 array([3.8573937], dtype=float32),
 array([3.3863063], dtype=float32),
 array([1.2452642], dtype=float32),
 array([1.52249], dtype=float32),
 array([1.7011094], dtype=float32),
 array([2.3914194], dtype=float32),
 array([0.98041], dtype=float32),
 array([0.9768834], dtype=float32),
 array([2.8450341], dtype=float32),
 array([1.0607189], dtype=float32),
 array([1.7255821], dtype=float32),
 array([4.1765666], dtype=float32),
 array([1.6945441], dtype=float32),
 array([1.3625596], dtype=float32),
 array([3.284602], dtype=float32)]

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [70]:
from sklearn import metrics
import math

In [118]:
rmse = math.sqrt(metrics.mean_squared_error(y_test, final_y_preds))
rmse

0.6395433125267445

# Great Job!